<a href="https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/beta/relax_amber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#relax your structure (using amber)

In [5]:
#@title install amber
from IPython.utils import io
import os
import subprocess
import tqdm.notebook
TQDM_BAR_FORMAT = '{l_bar}{bar}| {n_fmt}/{total_fmt} [elapsed: {elapsed} remaining: {remaining}]'
if not os.path.isfile("stereo_chemical_props.txt"):
  try:
    with tqdm.notebook.tqdm(total=100, bar_format=TQDM_BAR_FORMAT) as pbar:
      with io.capture_output() as captured:
        # Install py3dmol.
        %shell pip install py3dmol
        pbar.update(4)

        # Install OpenMM and pdbfixer.
        %shell rm -rf /opt/conda
        %shell wget -q -P /tmp \
          https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh \
            && bash /tmp/Miniconda3-latest-Linux-x86_64.sh -b -p /opt/conda \
            && rm /tmp/Miniconda3-latest-Linux-x86_64.sh
        pbar.update(15)

        PATH=%env PATH
        %env PATH=/opt/conda/bin:{PATH}
        %shell conda install -qy conda==4.13.0 \
            && conda install -qy -c conda-forge \
              python=3.7 \
              openmm=7.5.1 \
              pdbfixer
        pbar.update(80)

        %shell wget -q -P /content \
          https://git.scicore.unibas.ch/schwede/openstructure/-/raw/7102c63615b64735c4941278d92b554ec94415f8/modules/mol/alg/src/stereo_chemical_props.txt
        pbar.update(1)
  except subprocess.CalledProcessError:
    print(captured)
    raise

In [6]:
#@title install AlphaFold
import sys
if not os.path.isdir("/content/alphafold"):
  GIT_REPO = 'https://github.com/deepmind/alphafold'
  try:
    with tqdm.notebook.tqdm(total=100, bar_format=TQDM_BAR_FORMAT) as pbar:
      with io.capture_output() as captured:
        %shell rm -rf alphafold
        %shell git clone --branch main {GIT_REPO} alphafold
        pbar.update(10)
        # Install the required versions of all dependencies.
        %shell pip3 install -r ./alphafold/requirements.txt
        # Run setup.py to install only AlphaFold.
        %shell pip3 install --no-dependencies ./alphafold
        pbar.update(90)

        # Apply OpenMM patch.
        %shell pushd /opt/conda/lib/python3.7/site-packages/ && \
            patch -p0 < /content/alphafold/docker/openmm.patch && \
            popd

        # Make sure stereo_chemical_props.txt is in all locations where it could be searched for.
        %shell mkdir -p /content/alphafold/alphafold/common
        %shell cp -f /content/stereo_chemical_props.txt /content/alphafold/alphafold/common
        %shell mkdir -p /opt/conda/lib/python3.7/site-packages/alphafold/common/
        %shell cp -f /content/stereo_chemical_props.txt /opt/conda/lib/python3.7/site-packages/alphafold/common/

  except subprocess.CalledProcessError:
    print(captured)
    raise

if "/content/alphafold" not in sys.path:
  # Make sure everything we need is on the path.
  sys.path.append('/opt/conda/lib/python3.7/site-packages')
  sys.path.append('/content/alphafold')

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from alphafold.relax import relax
from alphafold.relax import utils
from alphafold.common import protein

def pdb_to_string(pdb_file):
  lines = []
  for line in open(pdb_file,"r"):
    if line[:6] == "HETATM" and line[17:20] == "MSE":
      line = "ATOM  "+line[6:17]+"MET"+line[20:]
    if line[:4] == "ATOM":
      lines.append(line)
  return "".join(lines)

def relax_me(pdb_in, pdb_out):
    protein_obj = protein.from_pdb_string(pdb_to_string(pdb_in))
    amber_relaxer = relax.AmberRelaxation(
      max_iterations=0,
      tolerance=2.39,
      stiffness=10.0,
      exclude_residues=[],
      max_outer_iterations=3,
      use_gpu=False)
    relaxed_pdb_lines, _, _ = amber_relaxer.process(prot=protein_obj)
    with open(pdb_out, 'w') as f:
        f.write(relaxed_pdb_lines)

In [24]:
relax_me(pdb_in="test_ce1eb_unrelaxed_rank_1_model_5.pdb",
         pdb_out="fix.pdb")